In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().resolve().parent  # parent of notebooks/
sys.path.append(str(project_root))

from src.data_enrichment import FinancialInclusionDataEnricher

In [2]:
DATA_PATH = project_root / "data" / "raw" / "ethiopia_fi_unified_data.csv"
REFERENCE_DATA_PATH = project_root / "data" / "raw" / "reference_codes.csv"

enricher = FinancialInclusionDataEnricher(
    data_path=DATA_PATH,
    reference_path=REFERENCE_DATA_PATH
)

In [3]:
enricher.load_data()

2026-01-30 12:32:39,376 | INFO | Loading unified dataset and reference codes
2026-01-30 12:32:39,393 | INFO | Dataset loaded: 43 rows


In [4]:
enricher.summarize_dataset()

2026-01-30 12:32:39,408 | INFO | Generating dataset summary


{'record_type': record_type
 observation    30
 event          10
 target          3
 Name: count, dtype: int64,
 'pillar': pillar
 ACCESS           16
 USAGE            11
 NaN              10
 GENDER            5
 AFFORDABILITY     1
 Name: count, dtype: int64,
 'source_type': source_type
 operator      15
 survey        10
 regulator      7
 research       4
 policy         3
 calculated     2
 news           2
 Name: count, dtype: int64,
 'confidence': confidence
 high      40
 medium     3
 Name: count, dtype: int64}

In [5]:
enricher.indicator_coverage()

,min,max,count
indicator_code,,,
ACC_OWNERSHIP,2014-12-31,2024-11-29,6
ACC_FAYDA,2024-08-15,2025-05-15,3
ACC_4G_COV,2023-06-30,2025-06-30,2
ACC_MM_ACCOUNT,2021-12-31,2024-11-29,2
GEN_GAP_ACC,2021-12-31,2024-11-29,2
USG_P2P_COUNT,2024-07-07,2025-07-07,2
ACC_MOBILE_PEN,2025-12-31,2025-12-31,1
GEN_GAP_MOBILE,2024-12-31,2024-12-31,1
GEN_MM_SHARE,2024-12-31,2024-12-31,1


In [6]:
enricher.add_observations()

2026-01-30 12:32:39,479 | INFO | Added 2 new observations


In [7]:
enricher.add_events()

2026-01-30 12:32:39,491 | INFO | Added 1 new events


In [8]:
enricher.add_impact_links()

2026-01-30 12:32:39,510 | INFO | Added 2 impact links


In [10]:
output_dir = project_root / "data" / "processed"

print (output_dir)

""" enricher.save_outputs(output_dir=output_dir) """

C:\Users\amare\Documents\KAIM\week-10\ethiopia-fi-forecast\data\processed


' enricher.save_outputs(output_dir=output_dir) '